In [ ]:
import underworld as uw
import glucifer
import numpy as np

uw.matplotlib_inline()
import matplotlib.pyplot as plt
plt.ion()

In [ ]:
annulus = uw.mesh._FeMesh_Annulus(elementRes=(20,100), radius=(3.0,6.), angularExtent=(0.0,360.0))
tField = uw.mesh.MeshVariable(annulus, nodeDofCount=1)

In [ ]:
fig = glucifer.Figure()
fig.append(glucifer.objects.Mesh(annulus))
fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True))
fig.show()

In [ ]:
# fig.append(glucifer.objects.Mesh(annulus))
# fig.append(glucifer.objects.Surface(annulus, tField, onMesh=True))
fig.show()

In [ ]:
# Set inner temperature BC to follow a cosine function and the outer surface to zero

outer = annulus.specialSets["MaxI_VertexSet"]
inner = annulus.specialSets["MinI_VertexSet"]

# Integer wave number
n = 3
# Min / Max Temperatures
t1 = 5.
t2 = 10.

for node in inner.data:
    theta = np.arctan(annulus.data[node][1] / annulus.data[node][0] )
    if annulus.data[node][0] < 0.:
        theta = np.pi - theta
    tField.data[node] = t2 + 1. * np.cos(n * theta)

tField.data[outer.data] = t1

In [ ]:
tBC = uw.conditions.DirichletCondition( variable=tField, indexSetsPerDof=(inner+outer))

In [ ]:
ssSLE = uw.systems.SteadyStateHeat(tField,fn_diffusivity=1.0, conditions=tBC)

In [ ]:
ssSolver = uw.systems.Solver(ssSLE)

In [ ]:
ssSolver.solve()

In [ ]:
fig.show()

In [ ]:
# Plot temperature as a function of angle, for a set of radii,
# then compare to an analytic solution
 
# Number of angles to plot
nP = 100
# Number of radii to plot
nR = 5

arrR = np.linspace(0.,annulus.radius[1]*0.99-annulus.radius[0],nP)
arrT = np.zeros(nP)
arrError = np.zeros(nP)
arrErrorSum = np.zeros(nR)
arrAnalytic = np.zeros(nP)
arrTheta = np.linspace(0.,2.,nP) * np.pi

# Parameters for the analytic solution
r2 = annulus.radius[0]
r1 = annulus.radius[1]
c = r2**(-n) / (1-r1**(2*n)*r2**(-2*n))
d = -c * r1**(2*n)

for r in np.linspace(annulus.radius[0],annulus.radius[1]*0.99,nR):
    if r == annulus.radius[0]:
        labelN = "Numerical"
        labelA = "Analytical"
    else:
        labelN = None
        labelA = None

    for i in range(nP):
        theta = arrTheta[i]
        pos = tuple(np.array( [ np.cos(theta),np.sin(theta)]) * r)

        t = tField.evaluate(pos)

        unperturb = np.log(r / r1) / np.log(r2/r1) * (t2-t1) + t1
        perturb = 1. * np.cos(n*theta)*(c * r **n + d * r ** (-n))

        tAn =  unperturb + perturb
         
        arrT[i] = t
        arrAnalytic[i] = tAn
        arrError[i] = (t-tAn)**2.

    print("RMS Error at r=%.2f is %.2e" %(r,np.sqrt(np.sum(arrError))))
    plt.plot(arrTheta,arrT,label=labelN)
    
    plotEvery = 4
    plt.scatter(arrTheta[::plotEvery],arrAnalytic[::plotEvery],label=labelA)
    
plt.ylabel("Temperature")
plt.xlabel("Angle")
plt.ylim(2.5,11.5)
plt.xlim(0,2.*np.pi)
plt.legend(loc='best')